In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from xgboost import XGBRegressor

In [2]:
df = pd.read_csv('final_df.csv')
df = df.drop(columns=['latitude','longitude'])
df

,Season,Date,hour,Stadium,HomeTeam,AwayTeam,is_weekend,is_holiday,is_close_match,HomeTeam_position,AwayTeam_position,is_top_match,has_min_one_top_club,had_extreme_weather,Attendance_percentage
0,2017_18,2017-11-04,15,John Smith's Stadium,Huddersfield Town,West Bromwich Albion,1,0,1,11,14,0,0,0,98.65
1,2017_18,2017-11-04,15,St James' Park,Newcastle United,AFC Bournemouth,1,0,0,9,19,0,0,0,99.81
2,2017_18,2017-11-04,15,St Mary's Stadium,Southampton,Burnley,1,0,0,9,7,0,0,1,94.15
3,2017_18,2017-11-04,15,Swansea.com Stadium,Swansea City,Brighton & Hove Albion,1,0,1,17,12,0,0,1,98.74
4,2017_18,2017-11-04,17,London Stadium,West Ham United,Liverpool,1,0,0,16,6,0,1,0,91.14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
752,2023_24,2024-03-30,20,Gtech Community Stadium,Brentford,Manchester United,1,0,0,16,6,0,1,0,99.35
753,2023_24,2024-03-30,15,Bramall Lane,Sheffield United,Fulham,1,0,0,19,12,0,0,0,91.77
754,2023_24,2024-03-30,15,Tottenham Hotspur Stadium,Tottenham Hotspur,Luton Town,1,0,0,5,18,0,1,0,97.91
755,2023_24,2024-03-31,14,Anfield,Liverpool,Brighton & Hove Albion,1,1,0,2,8,0,1,0,98.94


In [3]:
features = ['hour','is_weekend', 'is_holiday', 'is_close_match','HomeTeam_position', 'AwayTeam_position', 'is_top_match','has_min_one_top_club']
treatment = ['had_extreme_weather']
target = ['Attendance_percentage']

X_T = df[features + treatment]
y = df[target]

In [4]:
def plot_propensity_hist(X_T):
    plt.hist(X_T[X_T['T']==1]['propensity_score'], fc=(0, 0, 1, 0.5),bins=20,label='Treated')
    plt.hist(X_T[X_T['T']==0]['propensity_score'],fc=(1, 0, 0, 0.5),bins=20,label='Control')
    plt.legend()
    plt.xlabel('propensity score')
    plt.ylabel('number of units')
    plt.show()

# S-Learner

In [5]:
def calculate_ATE_S_learner(X_T, y):
    s_learner = XGBRegressor()
    # train
    s_learner.fit(X_T, y)

    X_T_all1 = X_T.copy()
    X_T_all1['had_extreme_weather'] = 1
    X_T_all0 = X_T.copy()
    X_T_all0['had_extreme_weather'] = 0

    # predict
    y_pred_all1 = s_learner.predict(X_T_all1)
    y_pred_all0 = s_learner.predict(X_T_all0)

    # calculate ATE
    ATE = np.mean(y_pred_all1 - y_pred_all0)
    return ATE

In [6]:
calculate_ATE_S_learner(X_T, y)

-0.41767216

# T-Learner

In [ ]:
def calculate_ATE_T_learner(X_T, y):
    t_learner_0 = XGBRegressor()
    t_learner_1 = XGBRegressor()
    # train
    t_learner_0.fit(X_T[X_T['had_extreme_weather'] == 0].drop(columns='had_extreme_weather'), y[X_T['had_extreme_weather'] == 0])
    t_learner_1.fit(X_T[X_T['had_extreme_weather'] == 1].drop(columns='had_extreme_weather'), y[X_T['had_extreme_weather'] == 1])

    # predict the treated
    y_hat_0 = t_learner_0.predict(X_T.drop(columns=['had_extreme_weather']))
    y_hat_1 = t_learner_1.predict(X_T.drop(columns=['had_extreme_weather']))

    # calculate ATT
    ATT = np.mean(y_hat_1 - y_hat_0)

    return ATT

In [ ]:
calculate_ATE_T_learner(X_T, y)

-0.6148347

# Matching

In [ ]:
from sklearn.covariance import EmpiricalCovariance
from sklearn.neighbors import NearestNeighbors

def calculate_ATT_matching(X_T, y):
    # match control to treated
    treated = X_T[X_T['had_extreme_weather'] == 1].copy()
    control = X_T[X_T['had_extreme_weather'] == 0].copy()

    # fit a nearest neighbor model
    nn = NearestNeighbors(n_neighbors=3, metric='euclidean')

    # fit the model
    nn.fit(control.drop(columns='T'))

    # find the nearest neighbors
    _, indices = nn.kneighbors(treated.drop(columns='T'))

    # calculate ATT
    ATT = np.mean(y[X_T['T'] == 1].values - y[X_T['T'] == 0].values[indices.flatten()].mean())
    return ATT

# IPW

In [ ]:
from sklearn.linear_model import LogisticRegression

def calculate_ATT_IPW(X_T, y):
    X = X_T.drop(columns='T')
    target = X_T['T']
    lr = LogisticRegression(max_iter=300, solver='lbfgs')
    lr.fit(X, target)
    propensity_scores = lr.predict_proba(X)[:, 1]
    X_T['propensity_score'] = propensity_scores
    
    # calculating first element of ATT
    elem_1 = np.mean(y[X_T['T'] == 1])

    # calculating second element of ATT
    propensity_scores_control = X_T[X_T['T'] == 0]['propensity_score']
    elem_2 = sum(y[X_T['T'] == 0]*propensity_scores_control/(1-propensity_scores_control))/sum(propensity_scores_control/(1-propensity_scores_control))

    # calculate ATT
    ATT = elem_1 - elem_2

    return ATT, X_T

In [ ]:
ATT_1, X_T_1_res = calculate_ATT_IPW(X_T_1.copy(), y_1)
ATT_2, X_T_2_res = calculate_ATT_IPW(X_T_2.copy(), y_2)
print(f'ATT for IPW: {ATT_1}')
print(f'ATT for IPW: {ATT_2}')
plot_propensity_hist(X_T_1_res)
plot_propensity_hist(X_T_2_res)

In [ ]:
from sklearn.calibration import calibration_curve

def plot_calibration_curve(data, title, ax):
    # Calculate calibration curve
    fraction_of_positives, mean_predicted_value = calibration_curve(data['T'], data['propensity_score'], n_bins=10)

    # Plot calibration curve
    ax.plot([0, 1], [0, 1], 'k:', label='Perfectly calibrated')
    ax.plot(mean_predicted_value, fraction_of_positives, 's-', label=title)
    ax.set_ylabel('Fraction of positives')
    ax.set_xlabel('Mean predicted value')
    ax.set_title(title)
    ax.legend()

# Create the figure and axes
fig, axes = plt.subplots(1, 2, figsize=(14, 7))  # 1 row, 2 columns

# Plot calibration curves for both datasets
plot_calibration_curve(X_T_1_res, 'Calibration Curve for Data1', axes[0])
plot_calibration_curve(X_T_2_res, 'Calibration Curve for Data2', axes[1])

# Adjust the layout
plt.tight_layout()

# Show the plot
plt.show()